In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('Delivery_Logistics.csv')
df.head()

,delivery_id,delivery_partner,package_type,vehicle_type,delivery_mode,region,weather_condition,distance_km,package_weight_kg,delivery_time_hours,expected_time_hours,delayed,delivery_status,delivery_rating,delivery_cost
0,250.99,delhivery,automobile parts,bike,same day,west,clear,297.0,46.96,1970-01-01 00:00:00.000000008,1970-01-01 00:00:00.000000008,no,delivered,3,1632.7206
1,250.99,xpressbees,cosmetics,ev van,express,central,cold,89.6,47.39,1970-01-01 00:00:00.000000002,1970-01-01 00:00:00.000000003,no,delivered,5,640.1700
2,250.99,shadowfax,groceries,truck,two day,east,rainy,273.5,26.89,1970-01-01 00:00:00.000000010,1970-01-01 00:00:00.000000016,no,delivered,4,1448.1700
3,250.99,dhl,electronics,ev van,same day,east,cold,269.7,12.69,1970-01-01 00:00:00.000000006,1970-01-01 00:00:00.000000008,no,delivered,3,1486.5700
4,250.99,dhl,clothing,van,two day,north,foggy,256.7,37.02,1970-01-01 00:00:00.000000009,1970-01-01 00:00:00.000000016,no,delivered,4,1394.5600


In [4]:
df.shape

(25000, 15)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   delivery_id          25000 non-null  float64
 1   delivery_partner     25000 non-null  object 
 2   package_type         25000 non-null  object 
 3   vehicle_type         25000 non-null  object 
 4   delivery_mode        25000 non-null  object 
 5   region               25000 non-null  object 
 6   weather_condition    25000 non-null  object 
 7   distance_km          25000 non-null  float64
 8   package_weight_kg    25000 non-null  float64
 9   delivery_time_hours  25000 non-null  object 
 10  expected_time_hours  25000 non-null  object 
 11  delayed              25000 non-null  object 
 12  delivery_status      25000 non-null  object 
 13  delivery_rating      25000 non-null  int64  
 14  delivery_cost        25000 non-null  float64
dtypes: float64(4), int64(1), object(10)


In [6]:
df.columns

Index(['delivery_id', 'delivery_partner', 'package_type', 'vehicle_type',
       'delivery_mode', 'region', 'weather_condition', 'distance_km',
       'package_weight_kg', 'delivery_time_hours', 'expected_time_hours',
       'delayed', 'delivery_status', 'delivery_rating', 'delivery_cost'],
      dtype='object')

In [15]:
#Perbaiki format waktu
df['delivery_time_hours'] = pd.to_datetime(df['delivery_time_hours'])
df['expected_time_hours'] = pd.to_datetime(df['expected_time_hours'])

# Pecah format tanggal dan jam pada expected time hours
df['expected_date'] = df['expected_time_hours'].dt.strftime('%d-%m-%Y')
df['expected_time'] = df['expected_time_hours'].dt.strftime('%H:%M:%S')

# Pecah format tanggal dan jam pada delivery time hours
df['delivery_date'] = df['delivery_time_hours'].dt.strftime('%d-%m-%Y')
df['delivery_time'] = df['delivery_time_hours'].dt.strftime('%H:%M:%S')

# Ubah kolom ke date time
df['expected_date'] = pd.to_datetime(df['expected_date'])
df['delivery_date'] = pd.to_datetime(df['delivery_date'])

# Create kolom tanggal keterlambatan
date_late = df['delivery_date'] - df['expected_date']
df['time_diff'] = date_late

# Tanda Kterlambatan
df['delay_flag'] = np.where(df['delayed'] == 'yes', 'late', 'ontime')

# Kategori Severity Delay
df['delay_category'] = pd.cut(df['time_diff'],
                              bins=[-999, 0, 1, 5, 999],
                              labels=['on time', '<1h late', '1–5h late', '>5h late'])

In [8]:
# Cek struktur Data
print(df.info())
print(df.describe())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   delivery_id          25000 non-null  float64        
 1   delivery_partner     25000 non-null  object         
 2   package_type         25000 non-null  object         
 3   vehicle_type         25000 non-null  object         
 4   delivery_mode        25000 non-null  object         
 5   region               25000 non-null  object         
 6   weather_condition    25000 non-null  object         
 7   distance_km          25000 non-null  float64        
 8   package_weight_kg    25000 non-null  float64        
 9   delivery_time_hours  25000 non-null  datetime64[ns] 
 10  expected_time_hours  25000 non-null  datetime64[ns] 
 11  delayed              25000 non-null  object         
 12  delivery_status      25000 non-null  object         
 13  delivery_rating 

In [16]:
# Top 5 Delivery Partner by Delay
df.groupby('delivery_partner')['delay_flag'].value_counts(normalize=True).unstack().sort_values('late', ascending=False).head()

delay_flag,late,ontime
delivery_partner,,
xpressbees,0.282732,0.717268
ekart,0.274188,0.725812
shadowfax,0.273026,0.726974
amazon logistics,0.272224,0.727776
blue dart,0.269478,0.730522


In [18]:
# Rata-rata delay per region
df.groupby('region')['time_diff'].mean().sort_values()

,time_diff
region,
central,0 days
east,0 days
north,0 days
south,0 days
west,0 days


In [20]:
# Rating per Partner
df.groupby('delivery_partner')['delivery_rating'].mean().sort_values(ascending=False)

,delivery_rating
delivery_partner,
fedex,3.698368
delhivery,3.687724
dhl,3.685225
blue dart,3.677627
ecom express,3.674504
shadowfax,3.661550
ekart,3.655480
amazon logistics,3.644412
xpressbees,3.608988


In [21]:
# Biaya vs Distance
df[['delivery_cost','distance_km']].corr()

,delivery_cost,distance_km
delivery_cost,1.000000,0.990772
distance_km,0.990772,1.000000


In [22]:
df.head()

,delivery_id,delivery_partner,package_type,vehicle_type,delivery_mode,region,weather_condition,distance_km,package_weight_kg,delivery_time_hours,expected_time_hours,delayed,delivery_status,delivery_rating,delivery_cost,expected_date,expected_time,delivery_date,delivery_time,date_late,time_late,time_diff,delay_flag,delay_category
0,250.99,delhivery,automobile parts,bike,same day,west,clear,297.0,46.96,1970-01-01 00:00:00.000000008,1970-01-01 00:00:00.000000008,no,delivered,3,1632.7206,1970-01-01,00:00:00,1970-01-01,00:00:00,0 days,0,0 days,ontime,on time
1,250.99,xpressbees,cosmetics,ev van,express,central,cold,89.6,47.39,1970-01-01 00:00:00.000000002,1970-01-01 00:00:00.000000003,no,delivered,5,640.1700,1970-01-01,00:00:00,1970-01-01,00:00:00,0 days,0,0 days,ontime,on time
2,250.99,shadowfax,groceries,truck,two day,east,rainy,273.5,26.89,1970-01-01 00:00:00.000000010,1970-01-01 00:00:00.000000016,no,delivered,4,1448.1700,1970-01-01,00:00:00,1970-01-01,00:00:00,0 days,0,0 days,ontime,on time
3,250.99,dhl,electronics,ev van,same day,east,cold,269.7,12.69,1970-01-01 00:00:00.000000006,1970-01-01 00:00:00.000000008,no,delivered,3,1486.5700,1970-01-01,00:00:00,1970-01-01,00:00:00,0 days,0,0 days,ontime,on time
4,250.99,dhl,clothing,van,two day,north,foggy,256.7,37.02,1970-01-01 00:00:00.000000009,1970-01-01 00:00:00.000000016,no,delivered,4,1394.5600,1970-01-01,00:00:00,1970-01-01,00:00:00,0 days,0,0 days,ontime,on time


In [23]:
df.to_csv('Delivery_Logistics_Clean.csv', index=False)